# Análisis de costos de cursos en el mercado

In [1]:
import pandas as pd
import numpy as np
import os
from hermetrics.levenshtein import Levenshtein

# Rutas de archivos

In [2]:
path = os.path.abspath(os.path.join(os.getcwd(), '..','2_data_understanding_module','collecting_initial_data','existing_data','DIRTY_DATA'))
path_additional_data = os.path.abspath(os.path.join(os.getcwd(), '..','2_data_understanding_module','collecting_initial_data','additional_data'))
path_dwh = os.path.abspath(os.path.join(os.getcwd(),'DATAWAREHOUSE'))
path_ppd = os.path.abspath(os.path.join(os.getcwd(),'CLEANED_DATA'))

In [3]:
# Cargar los archivos de costos de educacion continua, costos de cursos cec y cursos de OC, OEC y CI

df_dim_perfil = pd.read_csv(os.path.join(path_dwh,'datamart_oec','dim_perfil.csv'))
df_dim_curso = pd.read_csv(os.path.join(path_dwh,'datamart_ci','dim_curso.csv'))


### Palabras clave

In [4]:
keyword = ['Python', 'programación web', 'paginas web', 'datos', 'Excel', 'Word', 'power point', 'BI', 'inteligencia de negocios', 'powerBI', 
'big data', 'programación', 'marketing', 'web', 'html', 'css',  'java', 'javascript', 'base de datos', 'redes', 'telecomunicaciones', 'Android', 'IOS', 'ARDUINO'
'Business Intelligence', 
'Desarrollo de software',
'Fibra Óptica',
'Gerencia de sistemas',
'Hackeo Ético',
'Informática Forense',
'Linux', 'WINDOWS'
'Office',
'PHP',
'Project',
'software R', 'Tableu', 'rapidminer', 
'Mysql', 'sql', 'sqlserver', 'Oracle', 'postgresql',
'Seguridad informática',
'Aulas virtuales', 'Moodle', 'software',
'Wordpress', 'joomla', 'xoops', 'drupal'
]

keyword = np.char.lower(np.array(keyword))

In [4]:
lev = Levenshtein()

In [5]:
#Filtrar las areas de Tecnologia y TICs

df_dim_curso_perfil = pd.read_csv(os.path.join(path_dwh,'datamart_oc','dim_curso_perfil.csv'))
df_dim_curso_perfil = df_dim_curso_perfil[df_dim_curso_perfil['area_familia'] == 'TECNOLOGÍAS DE LA INFORMACIÓN Y COMUNICACIÓN']

In [29]:
df_coec = pd.read_csv(os.path.join(path_ppd,'costos_cursos_educacion_continua.csv'))
df_ccce = pd.read_csv(os.path.join(path_ppd,'costos_de_cursos_cec_epn.csv'))
df_coec = pd.concat([df_coec,df_ccce])
df_coec['curso'] = df_coec['curso'].str.cat(df_coec['modalidad'],sep=' | ')

In [30]:
data= df_dim_curso_perfil.groupby(['curso_perfil','modalidad']).count().reset_index()
data = data[['curso_perfil','modalidad']]
data['curso_perfil'] = data['curso_perfil'].str.cat(data['modalidad'],sep=' | ')

In [82]:
arr = []

for i in range(len(df_coec['curso'])):
    # Similaridad 
    sub_arr = []
    for j in range(len(data['curso_perfil'])):
        curso_fuente = df_coec['curso'].iloc[i]
        curso_comparar = data['curso_perfil'].iloc[j]
        similary = lev.similarity(curso_fuente,curso_comparar)
        aux_similary=0
        arr_curso = curso_comparar.split(' ')
        if curso_comparar in curso_fuente :
            similary = 1.0
        else:
            for word in arr_curso:
                if word in curso_fuente:
                    aux_similary+=1/len(arr_curso)
        if aux_similary > similary and aux_similary > 2/len(arr_curso):
            similary = aux_similary
        sub_arr.append(similary)
    arr.append(sub_arr)


In [83]:
res = pd.DataFrame(data=arr,index=df_coec['curso'],columns=data['curso_perfil'])


In [84]:
val = res[res['EXCEL BÁSICO | ONLINE']==res['EXCEL BÁSICO | ONLINE'].max()]['EXCEL BÁSICO | ONLINE'].index.values[0]
df_coec[df_coec['curso']==val]['precio_por_hora'].values[0]

3.208333333

In [48]:
df_dim_curso_perfil['curso_perfil_modalidad'] = df_dim_curso_perfil['curso_perfil'].str.cat(df_dim_curso_perfil['modalidad'],sep=' | ')

In [85]:
arr_curso_fuente = []
arr_precio_por_hora = []
for i in range(df_dim_curso_perfil.shape[0]):
    query = df_dim_curso_perfil['curso_perfil_modalidad'].iloc[i]
    val = res[res[query]==res[query].max()][query].index.values[0]
    arr_curso_fuente.append(df_coec[df_coec['curso']==val][['curso','precio_por_hora']].values[0])


In [86]:
aux_data = pd.DataFrame(data=arr_curso_fuente,columns=['curso_fuente','precio_por_hora'])
aux_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411 entries, 0 to 410
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   curso_fuente     411 non-null    object 
 1   precio_por_hora  411 non-null    float64
dtypes: float64(1), object(1)
memory usage: 6.5+ KB


In [103]:
res2 = pd.DataFrame(data=[df_dim_curso_perfil['curso_perfil_modalidad'].values,aux_data['curso_fuente'].values,aux_data['precio_por_hora'].values])
res2 = res2.transpose()
res2 = res2.rename(columns={0:'curso_perfil_modalidad',1:'curso_fuente',2:'precio_por_hora'})
#res2[res2['curso_fuente']=='PYTHON | PRESENCIAL']
#res2[res2['curso_perfil_modalidad'].str.contains('BI')]
#res2['curso_fuente'].value_counts()
'python' in 'tpythont' 

True

In [ ]:




# Realizar un procesado de texto en donde se contenga los nombres de los cursos por modalidad en caso de OC el resto de cursos solo se tomará un promedio general estimado de estos cursos

# Procesado de texto:
#   Aplicar a lista de las palabras clave de tecnologia
#   Validar si un curso es igual a la lista de costos de cursos
#   Buscar simetría en el nombre de los cursos en el sitio web y el nombre de los cursos de las listas de costos
#   Revisará los nombre de todos los cursos y determinará las palabras sin palabras_de_parada que se repitan más para obtener palabras clave en cuenta a cursos para buscarlas